In [1]:
import re
import datetime
import polars as pl

In [2]:
now = datetime.datetime.now()
print(now)

2024-11-03 03:58:23.936855


In [3]:
# set the threshold
threshold = 5

In [4]:
# polars method: https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.Expr.str.replace_all.html#polars-expr-str-replace-all

metadata = pl.read_csv(
    "../Data/Data_HNscore/HS_human_meta-data.csv",
    separator=","
).with_columns([
    pl.col("Cell-line(type)")
        .str.replace_all(",", " ")  # , to space
        .str.replace_all(r"\s+", " ")  # replace continuous spaces with one space
        .str.replace_all(r"[()]", "")  # remove parentheses (  )
        .str.replace_all(r"\s+$", "")  # remove trailing spaces ('\s+' matches any whitespace character, and '$' matches the end of the string)
        .str.replace_all(" ", "-")  # replace remaining spaces with hyphen
    ]
)
display(metadata)

SRA study count,SRA Study,pair,Source of Stress (Condition),Control,strategy,LibrarySelect,temperature,time,treatment,control temperature,Cell-line(type),tissue,Genotype/Phenotype,LibraryLayout,Instrument,RNA,note,
i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""SRP175158""",1,"""SRR8392699""","""SRR8392697""","""RNA-seq""","""cDNA""","""44℃""","""120min.""",null,null,"""human-lung-fibroblasts-WI38-ce…",null,"""young""","""SINGLE""","""Illumina HiSeq 2000""",null,"""Heat_shock_2hr RNA-seq""",null
null,null,2,"""SRR8392700""","""SRR8392698""","""RNA-seq""","""cDNA""","""44℃""","""120min.""",null,null,"""human-lung-fibroblasts-WI38-ce…",null,"""young""","""SINGLE""","""Illumina HiSeq 2000""",null,"""Heat_shock_2hr RNA-seq""",null
null,null,3,"""SRR8392703""","""SRR8392701""","""RNA-seq""","""cDNA""","""44℃""","""120min.""",null,null,"""human-lung-fibroblasts-WI38-ce…",null,"""senescent""","""SINGLE""","""Illumina HiSeq 2000""",null,null,null
null,null,4,"""SRR8392704""","""SRR8392702""","""RNA-seq""","""cDNA""","""44℃""","""120min.""",null,null,"""human-lung-fibroblasts-WI38-ce…",null,"""senescent""","""SINGLE""","""Illumina HiSeq 2000""",null,null,null
null,null,5,"""SRR8392707""","""SRR8392705""","""Ribo-seq""","""cDNA""","""44℃""","""120min.""",null,null,"""human-lung-fibroblasts-WI38-ce…",null,"""young""","""SINGLE""","""Illumina HiSeq 2000""",null,"""Heat_shock_2hr Ribo-seq""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,318,"""SRR5749643""","""SRR5749649""","""RNA-seq""","""cDNA""","""43""","""30min.""",null,null,"""colorectal-cancer-cell-line-HC…",null,null,"""PAIRED""","""Illumina HiSeq 2000""",null,"""2hr then heatshock for 0.5hr, …",null
null,null,319,"""SRR5749644""","""SRR5749650""","""RNA-seq""","""cDNA""","""43""","""30min.""",null,null,"""colorectal-cancer-cell-line-HC…",null,null,"""PAIRED""","""Illumina HiSeq 2000""",null,"""2hr then heatshock for 0.5hr, …",null
null,null,320,"""SRR5749645""","""SRR5749651""","""RNA-seq""","""cDNA""","""43""","""30min.""",null,null,"""colorectal-cancer-cell-line-HC…",null,null,"""PAIRED""","""Illumina HiSeq 2000""",null,"""2hr then heatshock for 0.5hr, …",null


In [5]:
HNratio = pl.read_csv(
    "../Data/Data_HNscore/HN-ratio_human_all.csv", 
    separator=","
).rename(
    {
        "GeneName": "GENEID"
    }
)


display(HNratio)

GENEID,SRR8392699,SRR8392700,SRR8392703,SRR8392704,SRR10129832,SRR9029562,SRR9029564,SRR9029566,SRR3309259,SRR3309260,SRR3309261,SRR7359773,SRR7359774,SRR7359775,SRR7359665,SRR7359666,SRR7359667,SRR7359701,SRR7359702,SRR7359703,SRR7359737,SRR7359738,SRR7359739,SRR12926169,SRR12926170,SRR12926171,SRR12926172,SRR12926185,SRR12926186,SRR12926187,SRR12926188,SRR12926161,SRR12926162,SRR12926163,SRR12926164,SRR12926177,…,SRR8669177,SRR8669167,SRR8669168,SRR8669169,SRR8669170,SRR8669171,SRR12267715,SRR12267716,SRR12267717,SRR12267718,SRR12267719,SRR12267720,SRR12267721,SRR12267722,SRR19077081,SRR19077089,SRR19077082,SRR19077090,SRR19077085,SRR19077093,SRR19077086,SRR19077094,SRR9257637,SRR9257638,SRR9257639,SRR9257640,SRR9257641,SRR9257642,SRR9257643,SRR9257644,SRR9257645,SRR12801712,SRR12801713,SRR12801714,SRR11067753,SRR11067754,SRR11067755
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""A1BG""",0.937577,0.90506,1.173431,0.951361,1.616084,1.587653,0.839417,0.707164,0.688915,0.807707,1.471826,1.149646,0.798949,0.84431,1.712014,1.676753,1.846766,0.763887,1.234583,1.154575,1.011787,1.123454,1.112957,0.292599,0.321263,0.486943,0.073171,3.063478,2.28231,2.732027,3.178165,2.342507,3.078054,0.913205,0.514791,1.535367,…,1.055852,0.874469,0.915239,0.7175,0.570618,1.264522,0.620855,1.955397,0.940095,1.337046,1.247015,0.376148,0.457872,3.581084,1.657477,1.321662,1.52167,1.234285,2.485797,1.422377,1.261323,0.989173,13.181142,12.866995,4.647388,4.738236,3.773852,0.150518,3.972351,1.388637,0.267622,1.0,1.0,1.0,1.0,1.0,0.497684
"""A1CF""",1.274827,1.0,1.0,1.0,1.578923,1.121376,1.125679,1.256608,0.867105,0.542718,1.223959,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.586528,1.0,1.224989,1.0,1.399655,1.183906,1.091216,1.0,0.851463,1.310149,1.386457,1.153764,1.0,1.0,1.0,0.698847,0.756844,1.159212,…,1.223524,0.989329,0.792394,0.905015,0.585017,1.272624,1.0,1.0,1.0,1.0,0.791308,1.0,1.0,1.0,0.942851,1.179185,1.37652,1.02926,2.009493,1.353698,1.637286,1.594538,1.0,1.0,1.29248,1.190399,1.0,1.0,1.0,1.0,1.0,0.939182,1.0,1.052121,1.029972,1.0,1.0
"""A2M""",0.954195,1.51097,1.340899,0.80041,2.867888,0.271588,1.02782,0.628021,0.646622,1.0,1.005016,0.947077,1.089874,1.046784,0.677007,0.429218,0.609098,1.001065,1.211348,1.411006,0.910106,0.946108,0.995809,0.990289,0.273583,0.782319,0.650255,1.087229,1.106072,3.679514,0.594014,1.751522,0.956616,0.712692,0.383038,0.613275,…,0.194784,1.0,1.0,1.327662,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.959312,2.021674,3.649038,2.879714,3.012148,2.617056,1.4001,1.383961,5.827531,2.253959,12.03367,1.0,1.31163,1.0,4.28526,1.0,1.0,1.0,1.0,1.0,0.875078,2.271058,1.53053
"""A2ML1""",1.078959,0.649897,0.789869,1.665942,2.674006,8.02823,3.515341,9.282938,0.929338,0.266727,1.754628,1.319572,2.897818,0.788736,2.818427,1.0,1.258707,0.966987,0.422454,1.879011,2.163199,0.642082,0.847668,0.530445,0.383827,0.258035,0.483833,3.024432,2.098743,2.784293,3.490833,1.084658,1.133235,0.25024,0.592231,1.586447,…,0.816836,0.780251,0.862604,1.054037,2.224936,1.060465,0.562087,5.225291,1.755832,1.573719,4.401472,0.18992,0.542405,0.394882,1.736568,2.135459,0.987405,1.331055,1.553919,0.814692,0.908963,0.885327,1.291847,2.489293,11.505186,0.566719,1.0,1.0,0.795947,1.0,1.0,1.0,1.0,1.0,2.026651,1.0,1.0
"""A3GALT2""",1.0,1.0,1.0,1.0,0.924578,1.0,1.0,0.468382,0.600246,0.241123,0.645082,1.0,2.618471,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.423799,2.104981,3.736439,0.276808,0.190093,6.835402,1.0,0.431012,2.228254,0.207173,1.0,1.008048,2.337433,…,1.544663,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.438702,1.0,1.0,1.0,1.0,0.554351,0.504145,0.983104,0.375045,1.990537,0.300613,0.218732,0.37912,1.0,1.0,3.475124,9.065566,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

In [6]:
rename_dict = dict(
    zip(
        metadata['Source of Stress (Condition)'], metadata['Cell-line(type)']
    )
)

print(rename_dict)

# change the column name based on the rename_dict
new_columns = []
seen = set()
for col in HNratio.columns:
    new_col = rename_dict.get(col, col)
    if new_col in seen:
        suffix = 1
        while f"{new_col}_{suffix}" in seen:
            suffix += 1
        new_col = f"{new_col}_{suffix}"
    seen.add(new_col)
    new_columns.append(new_col)

HNratio = HNratio.rename(
    dict(
        zip(
            HNratio.columns, new_columns
        )
    )
)

display(HNratio)

{'SRR8392699': 'human-lung-fibroblasts-WI38-cell', 'SRR8392700': 'human-lung-fibroblasts-WI38-cell', 'SRR8392703': 'human-lung-fibroblasts-WI38-cell', 'SRR8392704': 'human-lung-fibroblasts-WI38-cell', 'SRR8392707': 'human-lung-fibroblasts-WI38-cell', 'SRR8392708': 'human-lung-fibroblasts-WI38-cell', 'SRR8392711': 'human-lung-fibroblasts-WI38-cell', 'SRR8392712': 'human-lung-fibroblasts-WI38-cell', 'SRR2522096': 'WI38-lung-fibroblast-cells', 'SRR2522099': 'WI38-lung-fibroblast-cells', 'SRR2522102': 'WI38-lung-fibroblast-cells', 'SRR10129832': 'MCF7-mammary-gland-cell-culture', 'SRR13436541': 'MCF7-mammary-gland-cell-culture', 'SRR13436542': 'MCF7-mammary-gland-cell-culture', 'SRR13436543': 'MCF7-mammary-gland-cell-culture', 'SRR7309387': 'MCF7-mammary-gland-cell-culture', 'SRR9029562': 'OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells', 'SRR9029564': 'OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells', 'SRR9029566': 'OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells', 'SRR9029563': 'OCI-Ly1OCI-Ly1-di

GENEID,human-lung-fibroblasts-WI38-cell,human-lung-fibroblasts-WI38-cell_1,human-lung-fibroblasts-WI38-cell_2,human-lung-fibroblasts-WI38-cell_3,MCF7-mammary-gland-cell-culture,OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells,OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells_1,OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells_2,HEK293-cells,HEK293-cells_1,HEK293-cells_2,U2OS,U2OS_1,U2OS_2,U2OS_3,U2OS_4,U2OS_5,U2OS_6,U2OS_7,U2OS_8,U2OS_9,U2OS_10,U2OS_11,Colorectal-adenocarcinoma-DLD-1,Colorectal-adenocarcinoma-DLD-1_1,Colorectal-adenocarcinoma-DLD-1_2,Colorectal-adenocarcinoma-DLD-1_3,Colorectal-adenocarcinoma-DLD-1-cell-line,Colorectal-adenocarcinoma-DLD-1-cell-line_1,Colorectal-adenocarcinoma-DLD-1-cell-line_2,Colorectal-adenocarcinoma-DLD-1-cell-line_3,Colorectal-adenocarcinoma-DLD-1-cell-line_4,Colorectal-adenocarcinoma-DLD-1-cell-line_5,Colorectal-adenocarcinoma-DLD-1-cell-line_6,Colorectal-adenocarcinoma-DLD-1-cell-line_7,Colorectal-adenocarcinoma-DLD-1-cell-line_8,…,K562_26,K562_27,K562_28,K562_29,K562_30,K562_31,K562_32,K562_33,K562_34,K562_35,K562_36,K562_37,K562_38,K562_39,DLD-1-Colorectal-adenocarcinoma,DLD-1-Colorectal-adenocarcinoma_1,DLD-1-Colorectal-adenocarcinoma_2,DLD-1-Colorectal-adenocarcinoma_3,DLD-1-Colorectal-adenocarcinoma_4,DLD-1-Colorectal-adenocarcinoma_5,DLD-1-Colorectal-adenocarcinoma_6,DLD-1-Colorectal-adenocarcinoma_7,Primary-neurons_9,Primary-neurons_10,Primary-neurons_11,Primary-neurons_12,Primary-neurons_13,Primary-neurons_14,Primary-neurons_15,Primary-neurons_16,Primary-neurons_17,HeLa-cell-line_11,HeLa-cell-line_12,HeLa-cell-line_13,HeLa-cell-line_14,HeLa-cell-line_15,HeLa-cell-line_16
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""A1BG""",0.937577,0.90506,1.173431,0.951361,1.616084,1.587653,0.839417,0.707164,0.688915,0.807707,1.471826,1.149646,0.798949,0.84431,1.712014,1.676753,1.846766,0.763887,1.234583,1.154575,1.011787,1.123454,1.112957,0.292599,0.321263,0.486943,0.073171,3.063478,2.28231,2.732027,3.178165,2.342507,3.078054,0.913205,0.514791,1.535367,…,1.055852,0.874469,0.915239,0.7175,0.570618,1.264522,0.620855,1.955397,0.940095,1.337046,1.247015,0.376148,0.457872,3.581084,1.657477,1.321662,1.52167,1.234285,2.485797,1.422377,1.261323,0.989173,13.181142,12.866995,4.647388,4.738236,3.773852,0.150518,3.972351,1.388637,0.267622,1.0,1.0,1.0,1.0,1.0,0.497684
"""A1CF""",1.274827,1.0,1.0,1.0,1.578923,1.121376,1.125679,1.256608,0.867105,0.542718,1.223959,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.586528,1.0,1.224989,1.0,1.399655,1.183906,1.091216,1.0,0.851463,1.310149,1.386457,1.153764,1.0,1.0,1.0,0.698847,0.756844,1.159212,…,1.223524,0.989329,0.792394,0.905015,0.585017,1.272624,1.0,1.0,1.0,1.0,0.791308,1.0,1.0,1.0,0.942851,1.179185,1.37652,1.02926,2.009493,1.353698,1.637286,1.594538,1.0,1.0,1.29248,1.190399,1.0,1.0,1.0,1.0,1.0,0.939182,1.0,1.052121,1.029972,1.0,1.0
"""A2M""",0.954195,1.51097,1.340899,0.80041,2.867888,0.271588,1.02782,0.628021,0.646622,1.0,1.005016,0.947077,1.089874,1.046784,0.677007,0.429218,0.609098,1.001065,1.211348,1.411006,0.910106,0.946108,0.995809,0.990289,0.273583,0.782319,0.650255,1.087229,1.106072,3.679514,0.594014,1.751522,0.956616,0.712692,0.383038,0.613275,…,0.194784,1.0,1.0,1.327662,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.959312,2.021674,3.649038,2.879714,3.012148,2.617056,1.4001,1.383961,5.827531,2.253959,12.03367,1.0,1.31163,1.0,4.28526,1.0,1.0,1.0,1.0,1.0,0.875078,2.271058,1.53053
"""A2ML1""",1.078959,0.649897,0.789869,1.665942,2.674006,8.02823,3.515341,9.282938,0.929338,0.266727,1.754628,1.319572,2.897818,0.788736,2.818427,1.0,1.258707,0.966987,0.422454,1.879011,2.163199,0.642082,0.847668,0.530445,0.383827,0.258035,0.483833,3.024432,2.098743,2.784293,3.490833,1.084658,1.133235,0.25024,0.592231,1.586447,…,0.816836,0.780251,0.862604,1.05403

In [7]:
# define a function to categorize the HNratio
def categorize(x):
    if x >= threshold:
        return 'UP'
    elif x <= 1 / threshold:
        return 'DOWN'
    else:
        return 'UNCHANGED'

# apply the categorize function to all columns except GENEID
result = HNratio.with_columns([
    pl.col(col).map_elements(categorize, return_dtype=pl.String)
    for col in HNratio.columns if col != 'GENEID'
])

# display the result
display(result)

GENEID,human-lung-fibroblasts-WI38-cell,human-lung-fibroblasts-WI38-cell_1,human-lung-fibroblasts-WI38-cell_2,human-lung-fibroblasts-WI38-cell_3,MCF7-mammary-gland-cell-culture,OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells,OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells_1,OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells_2,HEK293-cells,HEK293-cells_1,HEK293-cells_2,U2OS,U2OS_1,U2OS_2,U2OS_3,U2OS_4,U2OS_5,U2OS_6,U2OS_7,U2OS_8,U2OS_9,U2OS_10,U2OS_11,Colorectal-adenocarcinoma-DLD-1,Colorectal-adenocarcinoma-DLD-1_1,Colorectal-adenocarcinoma-DLD-1_2,Colorectal-adenocarcinoma-DLD-1_3,Colorectal-adenocarcinoma-DLD-1-cell-line,Colorectal-adenocarcinoma-DLD-1-cell-line_1,Colorectal-adenocarcinoma-DLD-1-cell-line_2,Colorectal-adenocarcinoma-DLD-1-cell-line_3,Colorectal-adenocarcinoma-DLD-1-cell-line_4,Colorectal-adenocarcinoma-DLD-1-cell-line_5,Colorectal-adenocarcinoma-DLD-1-cell-line_6,Colorectal-adenocarcinoma-DLD-1-cell-line_7,Colorectal-adenocarcinoma-DLD-1-cell-line_8,…,K562_26,K562_27,K562_28,K562_29,K562_30,K562_31,K562_32,K562_33,K562_34,K562_35,K562_36,K562_37,K562_38,K562_39,DLD-1-Colorectal-adenocarcinoma,DLD-1-Colorectal-adenocarcinoma_1,DLD-1-Colorectal-adenocarcinoma_2,DLD-1-Colorectal-adenocarcinoma_3,DLD-1-Colorectal-adenocarcinoma_4,DLD-1-Colorectal-adenocarcinoma_5,DLD-1-Colorectal-adenocarcinoma_6,DLD-1-Colorectal-adenocarcinoma_7,Primary-neurons_9,Primary-neurons_10,Primary-neurons_11,Primary-neurons_12,Primary-neurons_13,Primary-neurons_14,Primary-neurons_15,Primary-neurons_16,Primary-neurons_17,HeLa-cell-line_11,HeLa-cell-line_12,HeLa-cell-line_13,HeLa-cell-line_14,HeLa-cell-line_15,HeLa-cell-line_16
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""A1BG""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""DOWN""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""",…,"""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""DOWN""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED"""
"""A1CF""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""",…,"""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""

In [8]:
# remove the suffix from the column name
def remove_suffix(col_name):
    return col_name.rsplit('_', 1)[0]


# get the sample column list
sample_columns = set(remove_suffix(col) for col in result.columns if col != 'GENEID')

print(sample_columns)

counts = []
for sample in sample_columns:
    # get the dataframe for each sample
    sample_df = result.select(
        ['GENEID'] + [col for col in result.columns if remove_suffix(col) == sample]
    )
    # count the number of UP, DOWN, UNCHANGED
    count = sample_df.with_columns([
        (pl.sum_horizontal([
            (pl.col(col) == "UP").cast(pl.Int32) for col in sample_df.columns if col != "GENEID"
        ])).alias(f"UP({sample})"),
        (pl.sum_horizontal([
            (pl.col(col) == "DOWN").cast(pl.Int32) for col in sample_df.columns if col != "GENEID"
        ])).alias(f"DOWN({sample})"),
        pl.sum_horizontal([
            (pl.col(col) == "UNCHANGED").cast(pl.Int32) for col in sample_df.columns if col != "GENEID"
        ]).alias(f"UNCHANGED({sample})")
    ])
    # append the count to the counts list
    counts.append(count)

# combine all samples
final_count = counts[0] # initialize the final_count with the first count
for count in counts[1:]: # [1:] is the list of counts from the second sample to the last sample
    final_count = final_count.join(
        count, 
        on=['GENEID'], 
        how='left',
        coalesce=True
    )

final_count = final_count.select(
    ['GENEID'] + [col for col in final_count.columns if col.startswith('UP(') or col.startswith('DOWN(') or col.startswith('UNCHANGED(')]
)

display(final_count)

{'U2OS', 'HEK293', 'human-foreskin-fibroblasts', 'Nasopharyngeal-epithelium-derived-cells-CNE2-cell', 'K562', 'neuronal-aggregates-day-50-derived-from-induced-pluripotent-stem-cells', 'colorectal-cancer-cell-line-HCT116', 'FaDu-spheroids-head-and-neck-squamous-cell-carcinomas-HNSCC', 'Colorectal-adenocarcinoma-DLD-1', 'MRC5-VA_CPSF3-dTAG-HA', 'cortical-brain-organoids', 'MRC5-VA', 'MCF7-mammary-gland-cell-culture', 'HeLa-cell-line', 'DLD-1-Colorectal-adenocarcinoma', 'HEPG2-cell', 'SW620', 'Primary-neurons', 'MRC5-VA-cell-line', 'Colorectal-adenocarcinoma-DLD-1-cell-line', 'OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells', "Raji-Burkitt's-B-lymphoma-cells-CDK9as", 'HEK293T', 'WI38-lung-fibroblast-cells', 'SASspheroids-head-and-neck-squamous-cell-carcinomas-HNSCC', 'K562-erythroleukemia-cells', 'MM.1S-myeloma-cell-line', 'human-lung-fibroblasts-WI38-cell', 'HEK293-cells', 'Human-Embryonic-Kidney-HEK-293T-Rex'}


GENEID,UP(U2OS),DOWN(U2OS),UNCHANGED(U2OS),UP(HEK293),DOWN(HEK293),UNCHANGED(HEK293),UP(human-foreskin-fibroblasts),DOWN(human-foreskin-fibroblasts),UNCHANGED(human-foreskin-fibroblasts),UP(Nasopharyngeal-epithelium-derived-cells-CNE2-cell),DOWN(Nasopharyngeal-epithelium-derived-cells-CNE2-cell),UNCHANGED(Nasopharyngeal-epithelium-derived-cells-CNE2-cell),UP(K562),DOWN(K562),UNCHANGED(K562),UP(neuronal-aggregates-day-50-derived-from-induced-pluripotent-stem-cells),DOWN(neuronal-aggregates-day-50-derived-from-induced-pluripotent-stem-cells),UNCHANGED(neuronal-aggregates-day-50-derived-from-induced-pluripotent-stem-cells),UP(colorectal-cancer-cell-line-HCT116),DOWN(colorectal-cancer-cell-line-HCT116),UNCHANGED(colorectal-cancer-cell-line-HCT116),UP(FaDu-spheroids-head-and-neck-squamous-cell-carcinomas-HNSCC),DOWN(FaDu-spheroids-head-and-neck-squamous-cell-carcinomas-HNSCC),UNCHANGED(FaDu-spheroids-head-and-neck-squamous-cell-carcinomas-HNSCC),UP(Colorectal-adenocarcinoma-DLD-1),DOWN(Colorectal-adenocarcinoma-DLD-1),UNCHANGED(Colorectal-adenocarcinoma-DLD-1),UP(MRC5-VA_CPSF3-dTAG-HA),DOWN(MRC5-VA_CPSF3-dTAG-HA),UNCHANGED(MRC5-VA_CPSF3-dTAG-HA),UP(cortical-brain-organoids),DOWN(cortical-brain-organoids),UNCHANGED(cortical-brain-organoids),UP(MRC5-VA),DOWN(MRC5-VA),UNCHANGED(MRC5-VA),…,UNCHANGED(Primary-neurons),UP(MRC5-VA-cell-line),DOWN(MRC5-VA-cell-line),UNCHANGED(MRC5-VA-cell-line),UP(Colorectal-adenocarcinoma-DLD-1-cell-line),DOWN(Colorectal-adenocarcinoma-DLD-1-cell-line),UNCHANGED(Colorectal-adenocarcinoma-DLD-1-cell-line),UP(OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells),DOWN(OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells),UNCHANGED(OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells),UP(Raji-Burkitt's-B-lymphoma-cells-CDK9as),DOWN(Raji-Burkitt's-B-lymphoma-cells-CDK9as),UNCHANGED(Raji-Burkitt's-B-lymphoma-cells-CDK9as),UP(HEK293T),DOWN(HEK293T),UNCHANGED(HEK293T),UP(WI38-lung-fibroblast-cells),DOWN(WI38-lung-fibroblast-cells),UNCHANGED(WI38-lung-fibroblast-cells),UP(SASspheroids-head-and-neck-squamous-cell-carcinomas-HNSCC),DOWN(SASspheroids-head-and-neck-squamous-cell-carcinomas-HNSCC),UNCHANGED(SASspheroids-head-and-neck-squamous-cell-carcinomas-HNSCC),UP(K562-erythroleukemia-cells),DOWN(K562-erythroleukemia-cells),UNCHANGED(K562-erythroleukemia-cells),UP(MM.1S-myeloma-cell-line),DOWN(MM.1S-myeloma-cell-line),UNCHANGED(MM.1S-myeloma-cell-line),UP(human-lung-fibroblasts-WI38-cell),DOWN(human-lung-fibroblasts-WI38-cell),UNCHANGED(human-lung-fibroblasts-WI38-cell),UP(HEK293-cells),DOWN(HEK293-cells),UNCHANGED(HEK293-cells),UP(Human-Embryonic-Kidney-HEK-293T-Rex),DOWN(Human-Embryonic-Kidney-HEK-293T-Rex),UNCHANGED(Human-Embryonic-Kidney-HEK-293T-Rex)
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""A1BG""",0,0,36,0,0,6,0,0,16,0,0,3,0,1,39,0,0,2,0,0,6,1,0,5,0,1,3,0,0,11,0,0,6,0,0,34,…,15,0,0,10,0,0,12,0,0,6,0,0,4,1,0,20,0,0,3,1,0,5,0,0,5,0,0,3,0,0,8,0,0,3,0,0,9
"""A1CF""",0,0,36,0,0,6,0,0,16,0,0,3,0,0,40,0,1,1,0,0,6,0,0,6,0,0,4,0,0,11,3,0,3,0,0,34,…,18,0,0,10,0,0,12,0,0,6,0,0,4,0,0,21,0,0,3,0,0,6,0,0,5,0,0,3,0,0,8,0,0,3,0,0,9
"""A2M""",0,0,36,0,0,6,0,0,16,0,0,3,2,1,37,0,0,2,1,0,5,0,0,6,0,0,4,0,1,10,0,0,6,2,1,31,…,10,0,0,10,0,0,12,0,0,6,0,0,4,1,0,20,0,0,3,0,0,6,0,0,5,0,0,3,0,0,8,0,0,3,0,0,9
"""A2ML1""",1,0,35,0,0,6,1,0,15,0,0,3,1,2,37,0,0,2,0,0,6,0,0,6,0,0,4,1,0,10,0,0,6,1,0,33,…,16,0,0,10,0,0,12,4,0,2,0,0,4,0,0,21,0,0,3,0,0,6,0,0,5,0,0,3,0,0,8,0,0,3,0,0,9
"""A3GALT2""",0,0,36,0,0,6,7,1,8,0,0,3,1,2,37,1,0,1,2,0,4,0,0,6,0,0,4,1,1,9,0,1,5,1,4,29,…,14,0,0,10,1,1,10,0,0,6,1,0,3,0,0,21,0,0,3,0,0,6,0,0,5,0,0,3,0,0,8,0,0,3,0,0,9
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZYG11A""",0,0,36,

In [9]:
# get the sample name list
sample_names = [col.split('(')[1].split(')')[0] for col in final_count.columns if col.startswith('UP(')]
print(sample_names)

# calculate HNscore for each sample
HNscore_sample_count = final_count.with_columns([
    (
        pl.col(f'UP({sample})') - pl.col(f'DOWN({sample})')
    ).alias(
        f'HN{threshold}({sample})'
    )
    for sample in sample_names # sample_names is the list of sample names
])

# select the columns that start with HN and end with the sample name
HNscore_sample_count = HNscore_sample_count.select(
    ['GENEID'] + 
    [col for col in HNscore_sample_count.columns if col.startswith(f'HN{threshold}(') and col.endswith(')')]
).with_columns(
    pl.sum_horizontal([
        pl.col(col) for col in HNscore_sample_count.columns if col.startswith(f'HN{threshold}(') and col.endswith(')')
    ]).alias(f'HN{threshold}(all)')
).sort(
    by=[f'HN{threshold}(all)'],
    descending=True
)

# save the result to csv
HNscore_sample_count.write_csv(
    f'../Data/Data_HNscore/HNscore_sample_count_HN{threshold}_human.tsv',
    separator='\t'
)

display(HNscore_sample_count.head())

['U2OS', 'HEK293', 'human-foreskin-fibroblasts', 'Nasopharyngeal-epithelium-derived-cells-CNE2-cell', 'K562', 'neuronal-aggregates-day-50-derived-from-induced-pluripotent-stem-cells', 'colorectal-cancer-cell-line-HCT116', 'FaDu-spheroids-head-and-neck-squamous-cell-carcinomas-HNSCC', 'Colorectal-adenocarcinoma-DLD-1', 'MRC5-VA_CPSF3-dTAG-HA', 'cortical-brain-organoids', 'MRC5-VA', 'MCF7-mammary-gland-cell-culture', 'HeLa-cell-line', 'DLD-1-Colorectal-adenocarcinoma', 'HEPG2-cell', 'SW620', 'Primary-neurons', 'MRC5-VA-cell-line', 'Colorectal-adenocarcinoma-DLD-1-cell-line', 'OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells', "Raji-Burkitt's-B-lymphoma-cells-CDK9as", 'HEK293T', 'WI38-lung-fibroblast-cells', 'SASspheroids-head-and-neck-squamous-cell-carcinomas-HNSCC', 'K562-erythroleukemia-cells', 'MM.1S-myeloma-cell-line', 'human-lung-fibroblasts-WI38-cell', 'HEK293-cells', 'Human-Embryonic-Kidney-HEK-293T-Rex']


GENEID,HN5(U2OS),HN5(HEK293),HN5(human-foreskin-fibroblasts),HN5(Nasopharyngeal-epithelium-derived-cells-CNE2-cell),HN5(K562),HN5(neuronal-aggregates-day-50-derived-from-induced-pluripotent-stem-cells),HN5(colorectal-cancer-cell-line-HCT116),HN5(FaDu-spheroids-head-and-neck-squamous-cell-carcinomas-HNSCC),HN5(Colorectal-adenocarcinoma-DLD-1),HN5(MRC5-VA_CPSF3-dTAG-HA),HN5(cortical-brain-organoids),HN5(MRC5-VA),HN5(MCF7-mammary-gland-cell-culture),HN5(HeLa-cell-line),HN5(DLD-1-Colorectal-adenocarcinoma),HN5(HEPG2-cell),HN5(SW620),HN5(Primary-neurons),HN5(MRC5-VA-cell-line),HN5(Colorectal-adenocarcinoma-DLD-1-cell-line),HN5(OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells),HN5(Raji-Burkitt's-B-lymphoma-cells-CDK9as),HN5(HEK293T),HN5(WI38-lung-fibroblast-cells),HN5(SASspheroids-head-and-neck-squamous-cell-carcinomas-HNSCC),HN5(K562-erythroleukemia-cells),HN5(MM.1S-myeloma-cell-line),HN5(human-lung-fibroblasts-WI38-cell),HN5(HEK293-cells),HN5(Human-Embryonic-Kidney-HEK-293T-Rex),HN5(all)
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""HSPA6""",36,0,16,3,15,2,6,6,4,8,4,31,5,4,8,2,12,16,10,12,6,4,18,3,6,0,3,4,0,1,245
"""HSPA1A""",24,0,16,3,19,0,6,6,4,8,5,31,5,2,8,2,12,11,10,12,6,4,13,3,6,4,3,8,0,8,239
"""HSPA1B""",24,0,16,3,19,0,6,6,4,8,5,31,5,2,8,2,12,15,10,12,4,4,14,3,6,3,2,5,1,1,231
"""DNAJB1""",24,0,16,3,9,0,6,6,4,8,5,24,5,2,8,2,12,9,10,12,4,4,15,2,6,3,2,6,3,0,210
"""BAG3""",24,0,16,3,7,0,6,1,4,8,3,22,4,1,6,2,12,7,8,12,6,4,10,3,6,0,2,4,0,0,181


&nbsp;

&nbsp;

## HN-score validation

In [10]:
# select HN5 columns (exclude HN5(all))
hn5_columns = [col for col in HNscore_sample_count.columns if col.startswith("HN5(") and col != "HN5(all)"]
display(hn5_columns)

['HN5(U2OS)',
 'HN5(HEK293)',
 'HN5(human-foreskin-fibroblasts)',
 'HN5(Nasopharyngeal-epithelium-derived-cells-CNE2-cell)',
 'HN5(K562)',
 'HN5(neuronal-aggregates-day-50-derived-from-induced-pluripotent-stem-cells)',
 'HN5(colorectal-cancer-cell-line-HCT116)',
 'HN5(FaDu-spheroids-head-and-neck-squamous-cell-carcinomas-HNSCC)',
 'HN5(Colorectal-adenocarcinoma-DLD-1)',
 'HN5(MRC5-VA_CPSF3-dTAG-HA)',
 'HN5(cortical-brain-organoids)',
 'HN5(MRC5-VA)',
 'HN5(MCF7-mammary-gland-cell-culture)',
 'HN5(HeLa-cell-line)',
 'HN5(DLD-1-Colorectal-adenocarcinoma)',
 'HN5(HEPG2-cell)',
 'HN5(SW620)',
 'HN5(Primary-neurons)',
 'HN5(MRC5-VA-cell-line)',
 'HN5(Colorectal-adenocarcinoma-DLD-1-cell-line)',
 'HN5(OCI-Ly1OCI-Ly1-diffuse-B-cell-lymphoma-cells)',
 "HN5(Raji-Burkitt's-B-lymphoma-cells-CDK9as)",
 'HN5(HEK293T)',
 'HN5(WI38-lung-fibroblast-cells)',
 'HN5(SASspheroids-head-and-neck-squamous-cell-carcinomas-HNSCC)',
 'HN5(K562-erythroleukemia-cells)',
 'HN5(MM.1S-myeloma-cell-line)',
 'HN5(huma

In [11]:
# count positive and negative samples
HNscore_sample_validate = HNscore_sample_count.with_columns([
    pl.fold(0, lambda acc, x: acc + (x > 0).cast(pl.Int32), hn5_columns).alias("positive_samples_count_human"),
    pl.fold(0, lambda acc, x: acc + (x < 0).cast(pl.Int32), hn5_columns).alias("negative_samples_count_human")
])

# assign tag to the contribution of samples
def tag_contribution(count, type, hn5_column):
    total_samples = len(hn5_column)
    if count == total_samples:
        return f"all samples {type} contribute to the score"
    else:
        return f"{count} / {total_samples} samples {type} contribute to the score"
    
HNscore_sample_validate = HNscore_sample_validate.with_columns([
    pl.lit(len(hn5_columns)).alias("total_samples"),
    pl.col("positive_samples_count_human").map_elements(lambda x: tag_contribution(x, "positively", hn5_columns), return_dtype=pl.Utf8).alias("positive_contribution_human"),
    pl.col("negative_samples_count_human").map_elements(lambda x: tag_contribution(x, "negatively", hn5_columns), return_dtype=pl.Utf8).alias("negative_contribution_human")
]
).sort(
    by=["HN5(all)"],
    descending=True
).drop(
    hn5_columns
)

# save the result to csv
HNscore_sample_validate.write_csv(
    f'../Data/Data_HNscore/HNscore_sample_validate_HN{threshold}_human.tsv',
    separator='\t'
)

display(HNscore_sample_validate.head())

GENEID,HN5(all),positive_samples_count_human,negative_samples_count_human,total_samples,positive_contribution_human,negative_contribution_human
str,i32,i32,i32,i32,str,str
"""HSPA6""",245,27,0,30,"""27 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""HSPA1A""",239,27,0,30,"""27 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""HSPA1B""",231,28,0,30,"""28 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""DNAJB1""",210,27,0,30,"""27 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""BAG3""",181,25,0,30,"""25 / 30 samples positively con…","""0 / 30 samples negatively cont…"


&nbsp;

&nbsp;


## Gene symbol checker

- ~~HGNCで念の為 `Approved symbol` をチェックしておく~~
- ~~[Multi-symbol checker](https://www.genenames.org/tools/multi-symbol-checker/) にアクセス~~
- TogoIDでヒットしたヒトのUniProt IDをキーにHGNC IDを取得しておく


In [12]:
# hgnc_current_symbol = pl.read_csv(
#     "../Data/Data_HNscore/hgnc-symbol-check.csv",
#     separator=","
# ).rename(
#     {
#         "Input" : "GENEID",
#         "Approved symbol" : "hgnc_symbol_id"
#     }
# ).with_columns(
#     [
#         pl.col("HGNC ID")
#             .str.extract(r":(\d+)$", 1)
#             .cast(pl.Int64)
#             .alias("hgnc_id")
#     ]
# ).select(
#     [
#         "GENEID",
#         "Match type",
#         "hgnc_id",
#         "hgnc_symbol_id"
#     ]
# )

# display(hgnc_current_symbol)

In [13]:
#upregulate rice gene
togoid_convert_to_human_up = pl.read_csv(
    "../out/rice_up/rice_up_togoid_convert.tsv",
    separator="\t"
).filter(
    pl.col("hgnc_id").is_not_null()
).rename(
    {
        "hgnc_symbol_id": "GENEID"
    }
)

display(togoid_convert_to_human_up)


uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,GENEID
str,str,str,str,i64,str
"""A0A024QZW4""","""ENSP00000518269""","""ENST00000710437""","""ENSG00000124786""",21601,"""SLC35B3"""
"""A0A024QZW4""","""ENSP00000498054""","""ENST00000648987""","""ENSG00000124786""",21601,"""SLC35B3"""
"""A0A024R3B9""","""ENSP00000435411""","""ENST00000525823""","""ENSG00000109846""",2389,"""CRYAB"""
"""A0A024R3B9""","""ENSP00000435960""","""ENST00000528961""","""ENSG00000109846""",2389,"""CRYAB"""
"""A0A024R3B9""","""ENSP00000435046""","""ENST00000533280""","""ENSG00000109846""",2389,"""CRYAB"""
…,…,…,…,…,…
"""X6RA14""","""ENSP00000367969""","""ENST00000378697""","""ENSG00000139684""",3465,"""ESD"""
"""X6RBJ8""","""ENSP00000380187""","""ENST00000396990""","""ENSG00000198876""",19911,"""DCAF12"""
"""X6RHX1""","""ENSP00000407993""","""ENST00000423670""","""ENSG00000000457""",19285,"""SCYL3"""


In [14]:
HNscore_up_sample_count_checked = togoid_convert_to_human_up.join(
    HNscore_sample_validate,
    on="GENEID",
    how="left",
    coalesce=True
).sort(
    by=[f'HN5(all)'],
    descending=True
).filter(
    pl.col("HN5(all)").is_not_null()
)

# save the result to csv
HNscore_up_sample_count_checked.write_csv(
    f'../Data/Data_HNscore/HNscore_sample_count_HN{threshold}_human_checked_up.tsv',
    separator='\t'
)

display(HNscore_up_sample_count_checked)


uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,GENEID,HN5(all),positive_samples_count_human,negative_samples_count_human,total_samples,positive_contribution_human,negative_contribution_human
str,str,str,str,i64,str,i32,i32,i32,i32,str,str
"""P17066""","""ENSP00000310219""","""ENST00000309758""","""ENSG00000173110""",5239,"""HSPA6""",245,27,0,30,"""27 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""P0DMV8""","""ENSP00000364802""","""ENST00000375651""","""ENSG00000204389""",5232,"""HSPA1A""",239,27,0,30,"""27 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""P0DMV8""","""ENSP00000382915""","""ENST00000400040""","""ENSG00000215328""",5232,"""HSPA1A""",239,27,0,30,"""27 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""P0DMV8""","""ENSP00000404524""","""ENST00000430065""","""ENSG00000235941""",5232,"""HSPA1A""",239,27,0,30,"""27 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""P0DMV8""","""ENSP00000408907""","""ENST00000433487""","""ENSG00000234475""",5232,"""HSPA1A""",239,27,0,30,"""27 / 30 samples positively con…","""0 / 30 samples negatively cont…"
…,…,…,…,…,…,…,…,…,…,…,…
"""A0A7P0T7D6""","""ENSP00000506550""","""ENST00000679536""","""ENSG00000119917""",5411,"""IFIT3""",-31,2,12,30,"""2 / 30 samples positively cont…","""12 / 30 samples negatively con…"
"""A0A7P0T855""","""ENSP00000504949""","""ENST00000680779""","""ENSG00000119917""",5411,"""IFIT3""",-31,2,12,30,"""2 / 30 samples positively cont…","""12 / 30 samples negatively con…"
"""O14879""","""ENSP00000360876""","""ENST00000371811""","""ENSG00000119917""",5411,"""IFIT3""",-31,2,12,30,"""2 / 30 samples positively cont…","""12 / 30 samples negatively con…"


&nbsp;

&nbsp;


In [15]:
# downregulate rice gene
togoid_convert_to_human_down = pl.read_csv(
    "../out/rice_down/rice_down_togoid_convert.tsv",
    separator="\t"
).filter(
    pl.col("hgnc_id").is_not_null()
).rename(
    {
        "hgnc_symbol_id": "GENEID"
    }
)

display(togoid_convert_to_human_down)


uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,GENEID
str,str,str,str,i64,str
"""A0A024QZP7""","""ENSP00000397973""","""ENST00000448257""","""ENSG00000170312""",1722,"""CDK1"""
"""A0A024QZW4""","""ENSP00000518269""","""ENST00000710437""","""ENSG00000124786""",21601,"""SLC35B3"""
"""A0A024QZW4""","""ENSP00000498054""","""ENST00000648987""","""ENSG00000124786""",21601,"""SLC35B3"""
"""A0A024R3M2""","""ENSP00000489896""","""ENST00000638157""","""ENSG00000023171""",29214,"""GRAMD1B"""
"""A0A024R5G9""","""ENSP00000435822""","""ENST00000526010""","""ENSG00000174080""",2531,"""CTSF"""
…,…,…,…,…,…
"""X6RGV9""","""ENSP00000403953""","""ENST00000420055""","""ENSG00000162639""",26400,"""HENMT1"""
"""X6RI37""","""ENSP00000408275""","""ENST00000451838""","""ENSG00000108061""",15454,"""SHOC2"""
"""X6RI37""","""ENSP00000510700""","""ENST00000689997""","""ENSG00000108061""",15454,"""SHOC2"""


In [16]:
HNscore_down_sample_count_checked = togoid_convert_to_human_down.join(
    HNscore_sample_validate,
    on="GENEID",
    how="left",
    coalesce=True
).sort(
    by=[f'HN5(all)'],
    descending=False
).filter(
    pl.col("HN5(all)").is_not_null()
)

# save the result to csv
HNscore_down_sample_count_checked.write_csv(
    f'../Data/Data_HNscore/HNscore_sample_count_HN{threshold}_human_checked_down.tsv',
    separator='\t'
)

display(HNscore_down_sample_count_checked)

uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,GENEID,HN5(all),positive_samples_count_human,negative_samples_count_human,total_samples,positive_contribution_human,negative_contribution_human
str,str,str,str,i64,str,i32,i32,i32,i32,str,str
"""Q96PN8""","""ENSP00000362634""","""ENST00000373534""","""ENSG00000162526""",15473,"""TSSK3""",-43,4,16,30,"""4 / 30 samples positively cont…","""16 / 30 samples negatively con…"
"""Q00444""","""ENSP00000309336""","""ENST00000312492""","""ENSG00000172789""",5127,"""HOXC5""",-29,5,6,30,"""5 / 30 samples positively cont…","""6 / 30 samples negatively cont…"
"""G3V3N9""","""ENSP00000451346""","""ENST00000554479""","""ENSG00000066629""",3330,"""EML1""",-23,3,7,30,"""3 / 30 samples positively cont…","""7 / 30 samples negatively cont…"
"""Q9UBP6""","""ENSP00000314441""","""ENST00000324871""","""ENSG00000037897""",7030,"""METTL1""",-23,2,7,30,"""2 / 30 samples positively cont…","""7 / 30 samples negatively cont…"
"""Q9UBP6""","""ENSP00000257848""","""ENST00000257848""","""ENSG00000037897""",7030,"""METTL1""",-23,2,7,30,"""2 / 30 samples positively cont…","""7 / 30 samples negatively cont…"
…,…,…,…,…,…,…,…,…,…,…,…
"""Q6P2M8""","""ENSP00000359169""","""ENST00000370150""","""ENSG00000130822""",13415,"""PNCK""",64,17,4,30,"""17 / 30 samples positively con…","""4 / 30 samples negatively cont…"
"""Q6P2M8""","""ENSP00000359161""","""ENST00000370142""","""ENSG00000130822""",13415,"""PNCK""",64,17,4,30,"""17 / 30 samples positively con…","""4 / 30 samples negatively cont…"
"""P56470""","""ENSP00000302100""","""ENST00000307751""","""ENSG00000171747""",6565,"""LGALS4""",70,15,5,30,"""15 / 30 samples positively con…","""5 / 30 samples negatively cont…"
